<a href="https://colab.research.google.com/github/BharatXBhalla/NLP-using-Tensorflow/blob/main/Embeddings/05_Movies_Reviews_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
tf.__version__

'2.7.0'

In [4]:
data,info = tfda.load("imdb_reviews",with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQQEAPK/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteQQEAPK/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
data

{'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>,
 'unsupervised': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>}

In [6]:
train_data,test_data = data['train'],data['test']

In [12]:
train_sentences =[]
train_labels=[]

test_sentences=[]
test_labels=[]

for sent,label in train_data:
  train_sentences.append(sent.numpy().decode('utf8'))
  train_labels.append(label.numpy())

for sent,label in test_data:
  test_sentences.append(sent.numpy().decode('utf8'))
  test_labels.append(label.numpy())

In [42]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [14]:
vocab_size = 10000 # No of Words
embedding_size = 16 # No of latent variables to reprenset words
max_length=150 # Max lenght of Sentence possible 
trunc_type ="post"
oov_tok="<oov>"

In [15]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs,maxlen=max_length,truncating=trunc_type)

In [39]:
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length,truncating=trunc_type)

In [18]:
word_index = tokenizer.word_index

In [22]:
len(list(word_index))

88583

In [29]:
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])

In [31]:
def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

In [32]:
decode_review(train_seqs[0])

"this was an absolutely terrible movie don't be <oov> in by christopher walken or michael <oov> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <oov> rebels were making their cases for <oov> maria <oov> <oov> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actor's like christopher <oov> good name i could barely sit through it"

In [33]:
train_sentences[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [35]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(input_length=max_length,input_dim=vocab_size,output_dim=embedding_size),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])

In [36]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 2400)              0         
                                                                 
 dense (Dense)               (None, 6)                 14406     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 174,413
Trainable params: 174,413
Non-trainable params: 0
_________________________________________________________________


In [38]:
num_epochs=10

In [43]:
model.fit(
    train_padded,
    train_labels,
    epochs=num_epochs,
    validation_data=(test_padded,test_labels)
)

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2199 - accuracy: 0.9186 - val_loss: 0.4045 - val_accuracy: 0.8248
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0851 - accuracy: 0.9795 - val_loss: 0.4816 - val_accuracy: 0.8265
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0273 - accuracy: 0.9957 - val_loss: 0.5531 - val_accuracy: 0.8240
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0123 - accuracy: 0.9982 - val_loss: 0.6131 - val_accuracy: 0.8235
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0067 - accuracy: 0.9991 - val_loss: 0.6750 - val_accuracy: 0.8194
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0029 - accuracy: 0.9996 - val_loss: 0.7261 - val_accuracy: 0.8192
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0011 - accuracy: 0.9999 - val_loss: 0.7930 - val_accuracy: 0.8174
Epoch 

In [44]:
l1 = model.layers[0]

In [50]:
embeddings = l1.get_weights()[0]

In [51]:
embeddings.shape

(10000, 16)

In [55]:
import io

vectors = io.open("vectors.tsv",'w',encoding='utf8')

meta = io.open("meta.tsv","w",encoding='utf8')

for i in range(1,vocab_size):
  word = reverse_word_index[i]
  vector = embeddings[i]
  meta.write(word+"\n")
  vectors.write("\t".join([str(x) for x in vector])+"\n")

In [56]:
vectors.close()
meta.close()

In [57]:
from google.colab import files
files.download("meta.tsv")
files.download("vectors.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>